# Imports

In [3]:
! pip install ultralytics
! pip install pycocotools
! pip install wandb


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pycocotools
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import yaml
import torch
import shutil
import wandb

%matplotlib inline

# Dataset Splitting

In [5]:
train_imgs_dir= r'E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\training_images'
train_labels= r'E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\train_solution_bounding_boxes (1).csv'

test_imgs_dir=r'E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\testing_images'

In [6]:
imgs_list=list(sorted(os.listdir(train_imgs_dir)))
idxs=list(range(len(imgs_list)))
np.random.shuffle(idxs)

train_idx=idxs[:int(0.8*len(idxs))]
val_idx=idxs[int(0.8*len(idxs)):]

# Custom Dataset to Yolo Format

The Ultralytics YOLO format is a dataset configuration format that allows you to define the dataset root directory, the relative paths to training/validation/testing image directories or *.txt files containing image paths, and a dictionary of class names.

In [7]:
# root directory
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data"

# images directory 
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images"

# train and test subdirectories with image directory
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images\train"
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images\val"

# labels directory
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels"

# train and test subdirectories with labels directory
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\train"
!mkdir "E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\val"


A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data already exists.
A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images already exists.
A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images\train already exists.


A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images\val already exists.
A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels already exists.
A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\train already exists.
A subdirectory or file E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\val already exists.


In [8]:
root_dir="E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data"
labels_dir="E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels"
images_dir="E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images"

In [9]:
df=pd.read_csv(train_labels)

width=676
height=380

df["class"]=0
df.rename(columns={'image':'img_name'}, inplace=True)

df["x_centre"]=(df["xmin"]+df["xmax"])/2
df["y_centre"]=(df["ymin"]+df["ymax"])/2
df["width"]=(df["xmax"]-df["xmin"])
df["height"]=(df["ymax"]-df["ymin"])

#normalizing bounding box coordinates
df["x_centre"]=df["x_centre"]/width
df["y_centre"]=df["y_centre"]/height
df["width"]=df["width"]/width
df["height"]=df["height"]/height

df_yolo=df[["img_name","class","x_centre","y_centre","width","height"]]
df_yolo.head()

,img_name,class,x_centre,y_centre,width,height
0,vid_4_1000.jpg,0,0.450434,0.539817,0.068741,0.095238
1,vid_4_10000.jpg,0,0.100217,0.557191,0.155572,0.129987
2,vid_4_10040.jpg,0,0.444645,0.543678,0.181621,0.157014
3,vid_4_10020.jpg,0,0.833213,0.531451,0.197540,0.155727
4,vid_4_10060.jpg,0,0.110347,0.559122,0.171491,0.136422


In [10]:
for idx,img_name in enumerate(imgs_list):
    subset="train"
    if idx in val_idx:
        subset="val"
        
    if np.isin(img_name,df_yolo["img_name"]):
        columns=["class","x_centre","y_centre","width","height"]
        img_bbox=df_yolo[df_yolo["img_name"]==img_name][columns].values
        
        label_file_path=os.path.join(labels_dir,subset,img_name[:-4]+".txt")
        with open(label_file_path,"w+") as f:
            for row in img_bbox:
                text=" ".join(row.astype(str))
                f.write(text)
                f.write("\n")
                
    old_image_path=os.path.join(train_imgs_dir,img_name)
    new_image_path=os.path.join(images_dir,subset,img_name)
    shutil.copy(old_image_path,new_image_path)

In [11]:
yolo_format=dict(path=r"E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data",
                 train=r"E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images\train",
                 val=r"E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\images\val",
                 nc=1,
                 names={0:"car"})
             
with open('E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

# Training

In [12]:
"""Device is determined automatically. If a GPU is available then it will be used, 
   otherwise training will start on CPU."""

model=YOLO('yolov8m.pt')
model.train(data="E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\yolo.yaml",epochs=50,patience=5,batch=8,
                    lr0=0.0005,imgsz=640)

New https://pypi.org/project/ultralytics/8.1.17 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.175  Python-3.11.2 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\yolo.yaml, epochs=50, patience=5, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_widt

Freezing layer 'model.22.dfl.conv.weight'
train: Scanning E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\train... 339 images, 621 backgrounds, 0 corrupt: 100%|██████████| 960/960 [00:01<00:00, 713.60it/s]
train: New cache created: E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\train.cache
val: Scanning E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\val... 137 images, 224 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<00:00, 965.19it/s]
val: New cache created: E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data\labels\val.cache
Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train5
Starting trai

KeyboardInterrupt: 

# Validate

In [ ]:
path_best_weights='E:\Code\Code_AI\FPT\DLP\Project_YOLO\Data\data'
model = YOLO(path_best_weights) 

metrics = model.val() 

In [ ]:
print(f"Mean Average Precision @.5:.95 : {metrics.box.map}")    
print(f"Mean Average Precision @ .50   : {metrics.box.map50}") 
print(f"Mean Average Precision @ .70   : {metrics.box.map75}")

# Prediction

In [ ]:
with torch.no_grad():
    results=model.predict(source=test_imgs_dir,conf=0.50,iou=0.75)

In [ ]:
# directory for saving predictions on test dataset
!mkdir "/kaggle/working/predictions"

In [ ]:
prediction_dir="/kaggle/working/predictions"

In [ ]:
test_img_list=[]
for result in results:
    if len(result.boxes.xyxy):
        name=result.path.split("/")[-1].split(".")[0]
        boxes=result.boxes.xyxy.cpu().numpy()
        scores=result.boxes.conf.cpu().numpy()
        
        test_img_list.append(name)
        
        label_file_path=os.path.join(prediction_dir,name+".txt")
        with open(label_file_path,"w+") as f:
            for score,box in zip(scores,boxes):
                text=f"{score:0.4f} "+" ".join(box.astype(str))
                f.write(text)
                f.write("\n")

# Prediction Visualization

In [ ]:
def show_bbox(img,boxes,scores,axis,color=(0,255,0)):
    boxes=boxes.astype(int)
    scores=scores
    img=img.copy()
    for i,box in enumerate(boxes):
        score=f"{scores[i]:.4f}"
        cv2.rectangle(img,(box[0],box[1]),(box[2],box[3]),color,2)
        y=box[1]-10 if box[1]-10>10 else box[1]+10
        cv2.putText(img,score,(box[0],y),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
    
    axis.imshow(img)
    axis.axis("off")

In [ ]:
fig,axes=plt.subplots(5,3,figsize=(12,12))
plt.subplots_adjust(wspace=0.1,hspace=0.1)
ax=axes.flatten()

imgs_name=np.random.choice(test_img_list,15)

for i,img_name in enumerate(imgs_name):
    img_file_path=os.path.join(test_imgs_dir,img_name+".jpg")
    img=cv2.imread(img_file_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    label_file_path=os.path.join(prediction_dir,img_name+".txt")
    label=pd.read_csv(label_file_path,sep=" ",header=None).values
    scores=label[:,0]
    boxes=label[:,1:]
    show_bbox(img,boxes,scores,axis=ax[i])
    
plt.savefig("1.png")